# Experimentos de EDAs sobre el problema Knapsack
Este notebook ejecuta y compara UMDA, MIMIC y un algoritmo genético básico sobre una instancia fija del problema Knapsack.
Se repiten los experimentos para obtener métricas de estabilidad como media y desviación estándar del mejor valor.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(42)
n_items = 50
weights = np.random.randint(1, 21, size=n_items)
values = np.random.randint(10, 101, size=n_items)
capacity = int(0.4 * np.sum(weights))

def knapsack_fitness(individual, weights, values, capacity):
    total_weight = np.sum(individual * weights)
    total_value = np.sum(individual * values)
    if total_weight > capacity:
        return 0
    return total_value

In [ ]:
def initialize_population(pop_size, n_bits):
    return np.random.randint(0, 2, size=(pop_size, n_bits))

def select_best(population, fitness, num_selected):
    idx = np.argsort(fitness)[-num_selected:]
    return population[idx]

def estimate_probabilities(selected):
    return np.mean(selected, axis=0)

def sample_from_probabilities(probabilities, pop_size):
    return (np.random.rand(pop_size, len(probabilities)) < probabilities).astype(int)

def run_umda_knapsack(fitness_fn, weights, values, capacity, n_bits=50, pop_size=100, selected_ratio=0.5, max_generations=100):
    population = initialize_population(pop_size, n_bits)
    history = []
    for generation in range(max_generations):
        fitness = np.array([fitness_fn(ind, weights, values, capacity) for ind in population])
        best = np.max(fitness)
        mean = np.mean(fitness)
        history.append((generation, best, mean))
        selected = select_best(population, fitness, int(pop_size * selected_ratio))
        probs = estimate_probabilities(selected)
        population = sample_from_probabilities(probs, pop_size)
    return pd.DataFrame(history, columns=["Generation", "Best", "Mean"])

In [ ]:
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1) - 1)
    return np.concatenate([parent1[:point], parent2[point:]])

def mutate(individual, mutation_rate=0.01):
    mutation = np.random.rand(len(individual)) < mutation_rate
    return np.bitwise_xor(individual, mutation.astype(int))

def run_ga_knapsack(fitness_fn, weights, values, capacity, n_bits=50, pop_size=100, selected_ratio=0.5, mutation_rate=0.01, max_generations=100):
    population = initialize_population(pop_size, n_bits)
    history = []
    for generation in range(max_generations):
        fitness = np.array([fitness_fn(ind, weights, values, capacity) for ind in population])
        best = np.max(fitness)
        mean = np.mean(fitness)
        history.append((generation, best, mean))
        num_selected = int(pop_size * selected_ratio)
        selected = select_best(population, fitness, num_selected)
        offspring = []
        while len(offspring) < pop_size:
            p1, p2 = selected[np.random.randint(num_selected)], selected[np.random.randint(num_selected)]
            child = crossover(p1, p2)
            child = mutate(child, mutation_rate)
            offspring.append(child)
        population = np.array(offspring)
    return pd.DataFrame(history, columns=["Generation", "Best", "Mean"])

In [ ]:
from collections import Counter
def conditional_entropy(x, y):
    joint_counts = Counter(zip(x, y))
    x_counts = Counter(x)
    total = len(x)
    cond_entropy = 0.0
    for (xi, yi), joint in joint_counts.items():
        px = x_counts[xi] / total
        pxy = joint / total
        pyx = pxy / px
        cond_entropy -= pxy * np.log2(pyx + 1e-12)
    return cond_entropy

def find_variable_order(selected):
    n_bits = selected.shape[1]
    remaining = list(range(n_bits))
    order = [remaining.pop(0)]
    while remaining:
        cond_entropies = [conditional_entropy(selected[:, i], selected[:, order[-1]]) for i in remaining]
        next_var = remaining[np.argmin(cond_entropies)]
        order.append(next_var)
        remaining.remove(next_var)
    return order

def build_conditional_probabilities(selected, order):
    cond_probs = {}
    for i, var in enumerate(order):
        if i == 0:
            cond_probs[var] = np.mean(selected[:, var])
        else:
            prev_var = order[i - 1]
            table = {}
            for v in [0, 1]:
                subset = selected[selected[:, prev_var] == v]
                table[v] = np.mean(subset[:, var]) if len(subset) > 0 else 0.5
            cond_probs[var] = table
    return cond_probs

def sample_conditional(cond_probs, order, pop_size):
    samples = np.zeros((pop_size, len(order)), dtype=int)
    for i in range(pop_size):
        for j, var in enumerate(order):
            if j == 0:
                samples[i, var] = np.random.rand() < cond_probs[var]
            else:
                prev_var = order[j - 1]
                prev_val = samples[i, prev_var]
                samples[i, var] = np.random.rand() < cond_probs[var][prev_val]
    return samples

def run_mimic_knapsack(fitness_fn, weights, values, capacity, n_bits=50, pop_size=100, selected_ratio=0.5, max_generations=100):
    population = initialize_population(pop_size, n_bits)
    history = []
    for generation in range(max_generations):
        fitness = np.array([fitness_fn(ind, weights, values, capacity) for ind in population])
        best = np.max(fitness)
        mean = np.mean(fitness)
        history.append((generation, best, mean))
        selected = select_best(population, fitness, int(pop_size * selected_ratio))
        order = find_variable_order(selected)
        cond_probs = build_conditional_probabilities(selected, order)
        population = sample_conditional(cond_probs, order, pop_size)
    return pd.DataFrame(history, columns=["Generation", "Best", "Mean"])

In [ ]:
def repeat_experiment_knapsack(run_function, name, n_repeats=10):
    all_runs = []
    for seed in range(n_repeats):
        np.random.seed(seed)
        df = run_function(knapsack_fitness, weights, values, capacity)
        df["Run"] = seed
        df["Algorithm"] = name
        all_runs.append(df)
    return pd.concat(all_runs, ignore_index=True)

umda_all = repeat_experiment_knapsack(run_umda_knapsack, "UMDA")
ga_all = repeat_experiment_knapsack(run_ga_knapsack, "GA")
mimic_all = repeat_experiment_knapsack(run_mimic_knapsack, "MIMIC")
all_data = pd.concat([umda_all, ga_all, mimic_all], ignore_index=True)
summary = all_data.groupby(["Algorithm", "Generation"]).agg(Best_mean=('Best', 'mean'), Best_std=('Best', 'std')).reset_index()
plt.figure(figsize=(10, 6))
for algo in summary["Algorithm"].unique():
    df = summary[summary["Algorithm"] == algo]
    plt.plot(df["Generation"], df["Best_mean"], label=algo)
    plt.fill_between(df["Generation"], df["Best_mean"] - df["Best_std"], df["Best_mean"] + df["Best_std"], alpha=0.2)
plt.title("Convergencia con desviación estándar (Knapsack)")
plt.xlabel("Generación")
plt.ylabel("Mejor valor")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()